<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия №3
<center>Автор материала: программист-исследователь Mail.Ru Group Юрий Кашницкий

# <center> Домашнее задание № 8
## <center> Vowpal Wabbit в задаче классификации тегов вопросов на Stackoverflow

## План
    1. Введение
    2. Описание данных
    3. Предобработка данных
    4. Обучение и проверка моделей
    5. Заключение

### 1. Введение

В этом задании вы будете делать примерно то же, что я каждую неделю –  в Mail.Ru Group: обучать модели на выборке в несколько гигабайт. Задание можно выполнить и на Windows с Python, но я рекомендую поработать под \*NIX-системой (например, через Docker) и активно использовать язык bash.
Немного снобизма (простите, но правда): если вы захотите работать в лучших компаниях мира в области ML, вам все равно понадобится опыт работы с bash под UNIX.

[Веб-форма](https://docs.google.com/forms/d/1VaxYXnmbpeP185qPk2_V_BzbeduVUVyTdLPQwSCxDGA/edit) для ответов.

Для выполнения задания понадобится установленный Vowpal Wabbit (уже есть в докер-контейнере курса, см. инструкцию в Wiki [репозитория](https://github.com/Yorko/mlcourse_open) нашего курса) и примерно 70 Гб дискового пространства. Я тестировал решение не на каком-то суперкомпе, а на Macbook Pro 2015 (8 ядер, 16 Гб памяти), и самая тяжеловесная модель обучалась около 12 минут, так что задание реально выполнить и с простым железом. Но если вы планируете когда-либо арендовать сервера Amazon, можно попробовать это сделать уже сейчас.

Материалы в помощь:
 - интерактивный [тьюториал](https://www.codecademy.com/en/courses/learn-the-command-line/lessons/environment/exercises/bash-profile) CodeAcademy по утилитам командной строки UNIX (примерно на час-полтора)
 - [статья](https://habrahabr.ru/post/280562/) про то, как арендовать на Amazon машину (еще раз: это не обязательно для выполнения задания, но будет хорошим опытом, если вы это делаете впервые)

### 2. Описание данных

Имеются 10 Гб вопросов со StackOverflow – [скачайте](https://drive.google.com/file/d/1ZU4J3KhJDrHVMj48fROFcTsTZKorPGlG/view) и распакуйте архив. 

Формат данных простой:<br>
<center>*текст вопроса* (слова через пробел) TAB *теги вопроса* (через пробел)

Здесь TAB – это символ табуляции.
Пример первой записи в выборке:

In [1]:
!head -1 ~/mlco_data/stackoverflow.10kk.tsv

head: cannot open '/home/kcostya/mlco_data/stackoverflow.10kk.tsv' for reading: No such file or directory


Здесь у нас текст вопроса, затем табуляция и теги вопроса: *css, css3* и *css-selectors*. Всего в выборке таких вопросов 10 миллионов. 

In [2]:
%%time
!wc -l ~/mlco_data/stackoverflow.10kk.tsv

wc: /home/kcostya/mlco_data/stackoverflow.10kk.tsv: No such file or directory
CPU times: user 4.93 ms, sys: 4.6 ms, total: 9.54 ms
Wall time: 113 ms


Обратите внимание на то, что такие данные я уже не хочу загружать в оперативную память и, пока можно, буду пользоваться эффективными утилитами UNIX –  head, tail, wc, cat, cut и прочими.

### 3. Предобработка данных

Давайте выберем в наших данных все вопросы с тегами *javascript, java, python, ruby, php, c++, c#, go, scala* и  *swift* и подготовим обучающую выборку в формате Vowpal Wabbit. Будем решать задачу 10-классовой классификации вопросов по перечисленным тегам.

Вообще, как мы видим, у каждого вопроса может быть несколько тегов, но мы упростим себе задачу и будем у каждого вопроса выбирать один из перечисленных тегов либо игнорировать вопрос, если таковых тегов нет. 
Но вообще VW поддерживает multilabel classification (аргумент  --multilabel_oaa).
<br>
<br>
Реализуйте в виде отдельного файла `preprocess.py` код для подготовки данных. Он должен отобрать строки, в которых есть перечисленные теги, и переписать их в отдельный файл в формат Vowpal Wabbit. Детали:
 - скрипт должен работать с аргументами командной строки: с путями к файлам на входе и на выходе
 - строки обрабатываются по одной (можно использовать tqdm для подсчета числа итераций)
 - если табуляций в строке нет или их больше одной, считаем строку поврежденной и пропускаем
 - в противном случае смотрим, сколько в строке тегов из списка *javascript, java, python, ruby, php, c++, c#, go, scala* и  *swift*. Если ровно один, то записываем строку в выходной файл в формате VW: `label | text`, где `label` – число от 1 до 10 (1 - *javascript*, ... 10 – *swift*). Пропускаем те строки, где интересующих тегов больше или меньше одного 
 - из текста вопроса надо выкинуть двоеточия и вертикальные палки, если они есть – в VW это спецсимволы

In [1]:
import os
from tqdm import tqdm
from time import time
import numpy as np
from sklearn.metrics import accuracy_score
import itertools

Должно получиться вот такое число строк – 4389054. 10 Гб у меня обработались примерно за 2 минуты.

In [4]:
%%writefile ~/mlco_data/preprocess.py
import sys
from tqdm import tqdm

topics = ['javascript', 'java', 'python', 'ruby', 'php',
          'c++', 'c#', 'go', 'scala', 'swift']
topic_set = set(topics)
topic_map = dict(zip(topics, range(1, len(topics) + 1)))


num_corrupted, num_selected = 0, 0
with open(sys.argv[1]) as inp_file, open(sys.argv[2], 'w') as out_file:
    for line in tqdm(inp_file):
        values = line.strip().split('\t')
        if len(values) != 2:
            num_corrupted += 1
            continue
        text, labels = values
        labels = set(labels.split())
        topics_from_list = labels.intersection(topic_set)
        if len(topics_from_list) == 1:
            num_selected += 1
            out_file.write('{} | {}\n'.format(str(topic_map[list(topics_from_list)[0]]), 
                                              text.strip().replace(':', '').replace('|', '')))
print("{} lines selected, {} lines corrupted.".format(num_selected, num_corrupted))

Overwriting /home/kcostya/mlco_data/preprocess.py


In [5]:
!python -m py_compile ~/mlco_data/preprocess.py

In [6]:
!python3.5 ~/mlco_data/preprocess.py ~/mlco_data/stackoverflow.10kk.tsv ~/mlco_data/stackoverflow.vw

Traceback (most recent call last):
  File "/home/kcostya/mlco_data/preprocess.py", line 11, in <module>
    with open(sys.argv[1]) as inp_file, open(sys.argv[2], 'w') as out_file:
FileNotFoundError: [Errno 2] No such file or directory: '/home/kcostya/mlco_data/stackoverflow.10kk.tsv'


In [7]:
%%time
!wc -l ~/mlco_data/stackoverflow.vw

4389054 /home/kcostya/mlco_data/stackoverflow.vw
CPU times: user 328 ms, sys: 95.9 ms, total: 424 ms
Wall time: 12.9 s


In [9]:
!gzip ~/mlco_data/stackoverflow.10kk.tsv

Поделите выборку на обучающую, проверочную и тестовую части в равной пропорции - по  1463018 в каждый файл. Перемешивать не надо, первые 1463018 строк должны пойти в обучающую часть `stackoverflow_train.vw`, последние 1463018 – в тестовую `stackoverflow_test.vw`, оставшиеся – в проверочную `stackoverflow_valid.vw`. 

Также сохраните векторы ответов для проверочной и тестовой выборки в отдельные файлы `stackoverflow_valid_labels.txt` и `stackoverflow_test_labels.txt`.

Тут вам помогут утилиты `head`, `tail`, `split`, `cat` и `cut`.

In [21]:
%%time
!split -l 1463018 ~/mlco_data/stackoverflow.vw ~/mlco_data/new

CPU times: user 960 ms, sys: 238 ms, total: 1.2 s
Wall time: 47.6 s


In [22]:
%%time
!mv ~/mlco_data/newaa ~/mlco_data/stackoverflow_train.vw
!mv ~/mlco_data/newab ~/mlco_data/stackoverflow_test.vw
!mv ~/mlco_data/newac ~/mlco_data/stackoverflow_valid.vw

CPU times: user 8.6 ms, sys: 9.77 ms, total: 18.4 ms
Wall time: 329 ms


In [37]:
%%time
!cut -f 1 -d '|' ~/mlco_data/stackoverflow.vw > ~/mlco_data/stackoverflow_labels.txt
!cut -f 1 -d '|' ~/mlco_data/stackoverflow_test.vw > ~/mlco_data/stackoverflow_test_labels.txt
!cut -f 1 -d '|' ~/mlco_data/stackoverflow_valid.vw > ~/mlco_data/stackoverflow_valid_labels.txt

CPU times: user 380 ms, sys: 116 ms, total: 496 ms
Wall time: 19.5 s


In [35]:
!head -2 ~/mlco_data/stackoverflow.vw

1 | i ve got some code in window scroll that checks if an element is visible then triggers another function however only the first section of code is firing both bits of code work in and of themselves if i swap their order whichever is on top fires correctly my code is as follows fn isonscreen function use strict var win window viewport top win scrolltop left win scrollleft bounds this offset viewport right viewport left + win width viewport bottom viewport top + win height bounds right bounds left + this outerwidth bounds bottom bounds top + this outerheight return viewport right lt bounds left viewport left gt bounds right viewport bottom lt bounds top viewport top gt bounds bottom window scroll function use strict var load_more_results ajax load_more_results isonscreen if load_more_results true loadmoreresults var load_more_staff ajax load_more_staff isonscreen if load_more_staff true loadmorestaff what am i doing wrong can you only fire one event from window scroll i assume not
4 |

In [36]:
!head -2 ~/mlco_data/y_total.txt

1 
4 


### 4. Обучение и проверка моделей

Обучите Vowpal Wabbit на выборке `stackoverflow_train.vw` 9 раз, перебирая параметры passes (1,3,5), ngram (1,2,3).
Остальные параметры укажите следующие: `loss_function=hinge`, `bit_precision`=28 и `seed`=17. Также скажите VW, что это 10-классовая задача.

Проверяйте долю правильных ответов на выборке `stackoverflow_valid.vw`. Выберите лучшую модель и проверьте качество на выборке `stackoverflow_test.vw`.

In [2]:
PATH_TO_DATA = '~/mlco_data/'

In [13]:
parameter_grid = list(itertools.product((1,3,5), (1,2,3)))
parameter_grid

[(1, 1), (1, 2), (1, 3), (3, 1), (3, 2), (3, 3), (5, 1), (5, 2), (5, 3)]

In [14]:
for i in range(len(parameter_grid)):
    print(parameter_grid[i][0], parameter_grid[i][1])

1 1
1 2
1 3
3 1
3 2
3 3
5 1
5 2
5 3


In [11]:
%%time

accuracy_scores = []  

for i in range(len(parameter_grid)):
    
    passes = parameter_grid[i][0]
    ngram = parameter_grid[i][1]
    
    !vw -d $PATH_TO_DATA/stackoverflow_train.vw \
    --loss_function hinge --bit_precision 28 --random_seed 17 --oaa 10 -c \
    --passes {passes} --ngram {ngram} \
    -f $PATH_TO_DATA/stackoverflow_model_{i}.vw --quiet

    !vw -t -i $PATH_TO_DATA/stackoverflow_model_{i}.vw \
    -d $PATH_TO_DATA/stackoverflow_valid.vw \
    -p $PATH_TO_DATA/stackoverflow_valid_pred_{i}.csv --random_seed 17 --quiet
    
    vw_pred = np.loadtxt('../../../mlco_data/stackoverflow_valid_pred_{}.csv'.format(i))
    test_labels = np.loadtxt('../../../mlco_data/stackoverflow_valid_labels.txt')
    accuracy_scores = accuracy_scores.append(accuracy_score(test_labels, vw_pred))

with open('../../../mlco_data/accuracy_scores_valid.txt', w) as file:
    file.write(accuracy_scores)

CPU times: user 3min 46s, sys: 29.8 s, total: 4min 16s
Wall time: 1h 40min 45s


In [26]:
print("Best score:{} \nBest parameters:{}.".format(round(max(accuracy_scores),3), 
                                                   parameter_grid[accuracy_scores.index(max(accuracy_scores))]))

Best score:7 
Best parameters:(5, 1).


In [9]:
best_model_index = accuracy_scores.index(max(accuracy_scores))

In [10]:
!vw -t -i $PATH_TO_DATA/stackoverflow_model_{best_model_index}.vw \
-d $PATH_TO_DATA/stackoverflow_test.vw \
-p $PATH_TO_DATA/stackoverflow_test_pred.csv --random_seed 17 --quiet

In [11]:
vw_pred = np.loadtxt('../../../mlco_data/stackoverflow_test_pred.csv')
test_labels = np.loadtxt('../../../mlco_data/stackoverflow_test_labels.txt')

In [12]:
accuracy_score(test_labels, vw_pred)

0.92997010289688853

<font color='red'> Вопрос 1.</font> Какое сочетание параметров дает наибольшую долю правильных ответов на проверочной выборке `stackoverflow_valid.vw`?
- Биграммы и 3 прохода по выборке
- Триграммы и 5 проходов по выборке
- Биграммы и 1 проход по выборке
- Униграммы и 1 проход по выборке

Проверьте лучшую (по доле правильных ответов на валидации) модель на тестовой выборке. 

In [ ]:
''' ВАШ КОД ЗДЕСЬ '''

<font color='red'> Вопрос 2.</font> Как соотносятся доли правильных ответов лучшей (по доле правильных ответов на валидации) модели на проверочной и на тестовой выборках? (здесь % – это процентный пункт, т.е., скажем, снижение с 50% до 40% – это на 10%, а не 20%).
- На тестовой ниже примерно на 2%
- На тестовой ниже примерно на 3%
- Результаты почти одинаковы – отличаются меньше чем на 0.5%

Обучите VW с параметрами, подобранными на проверочной выборке, теперь на объединении обучающей и проверочной выборок. Посчитайте долю правильных ответов на тестовой выборке. 

In [ ]:
''' ВАШ КОД ЗДЕСЬ '''

<font color='red'> Вопрос 3.</font> На сколько процентных пунктов повысилась доля правильных ответов модели после обучения на вдвое большей выборке (обучающая `stackoverflow_train.vw` + проверочная `stackoverflow_valid.vw`) по сравнению с моделью, обученной только на `stackoverflow_train.vw`?
 - 0.1%
 - 0.4%
 - 0.8%
 - 1.2%

### 5. Заключение

В этом задании мы только познакомились с Vowpal Wabbit. Что еще можно попробовать:
 - Классификация с несколькими ответами (multilabel classification, аргумент  `multilabel_oaa`) – формат данных тут как раз под такую задачу
 - Настройка параметров VW с hyperopt, авторы библиотеки утверждают, что качество должно сильно зависеть от параметров изменения шага градиентного спуска (`initial_t` и `power_t`). Также можно потестировать разные функции потерь – обучать логистическую регресиию или линейный SVM
 - Познакомиться с факторизационными машинами и их реализацией в VW (аргумент `lrq`)